In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
#a) Create Airline Pysaprk sql dataFrame.
airline = spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv", header=True, inferSchema=True)

In [0]:
#b) Describe the table schema &amp; show 10 rows of Dataset
airline.printSchema()
airline.show(10, truncate=False)

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

####c) Average arrival delay caused by airlines.

In [0]:
airline.createOrReplaceTempView('Airlines_Data')
spark.sql("select * from Airlines_Data ").show(1 , truncate=True)

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
spark.sql("select AIRLINE, avg(ARRIVAL_DELAY) from Airlines_Data group by AIRLINE").show()

+-------+------------------+
|AIRLINE|avg(ARRIVAL_DELAY)|
+-------+------------------+
|     UA| 6.697221614526362|
|     NK|14.206426484907498|
|     AA| 8.386631979187513|
|     EV|10.884270870655678|
|     B6| 13.95852534562212|
|     DL|2.8144726712856043|
|     OO|10.154792043399638|
|     F9|24.103448275862068|
|     US| 5.977315185481719|
|     MQ|19.231592604605904|
|     HA| 4.072423398328691|
|     AS|-1.531766200762389|
|     VX| 5.128571428571429|
|     WN| 3.697840458351697|
+-------+------------------+



In [0]:
#d)Days of months with average Arrival delays.
spark.sql("select DAY, avg(ARRIVAL_DELAY) from Airlines_Data group by DAY").show()

+---+------------------+
|DAY|avg(ARRIVAL_DELAY)|
+---+------------------+
| 31|-1.196594427244582|
| 28| 3.257425742574257|
| 27| 4.706711409395973|
| 26| 11.96778269109286|
| 12| 11.24892703862661|
| 22| 6.550920245398773|
|  1|14.807807807807809|
| 13|3.3769751693002257|
|  6|10.608832807570979|
| 16| 9.124321062160531|
|  3|18.141541038525965|
| 20|3.8770149253731345|
|  5| 16.23861262014208|
| 19|1.6344282238442822|
| 15| 2.966753585397653|
|  9| 4.421887390959556|
| 17| 8.761435608726249|
|  4|17.157790927021697|
|  8| 5.232349165596919|
| 23| 4.207086133170434|
+---+------------------+
only showing top 20 rows



In [0]:
#e)Days of months with median Departure delays.
spark.sql("select DAY,percentile_approx(DEPARTURE_DELAY, 0.5) as median_departure from Airlines_Data group by DAY order by median_departure").show()

+---+----------------+
|DAY|median_departure|
+---+----------------+
| 30|              -3|
| 29|              -3|
| 31|              -3|
| 20|              -2|
| 21|              -2|
| 19|              -2|
| 27|              -2|
| 24|              -2|
| 11|              -2|
| 14|              -2|
| 18|              -2|
|  7|              -2|
| 10|              -2|
| 28|              -2|
| 25|              -2|
| 26|              -1|
| 22|              -1|
| 13|              -1|
| 15|              -1|
| 16|              -1|
+---+----------------+
only showing top 20 rows



In [0]:
#f) Arrange weekdays with respect to the average arrival delays.
spark.sql("select DAY_OF_WEEK, avg(ARRIVAL_DELAY) from Airlines_Data group by DAY_OF_WEEK order by avg(ARRIVAL_DELAY)").show()

+-----------+------------------+
|DAY_OF_WEEK|avg(ARRIVAL_DELAY)|
+-----------+------------------+
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          2| 8.033644102148358|
|          7|10.110840438489646|
|          1|10.807447207297264|
+-----------+------------------+



In [0]:
#g) Show Analysis for each month with total number of cancellations. (Descending).
spark.sql("SELECT MONTH,SUM(CANCELLED) as total_cancellations from Airlines_Data GROUP BY MONTH order by total_cancellations desc").show()

+-----+-------------------+
|MONTH|total_cancellations|
+-----+-------------------+
|    2|               1087|
|    1|                608|
|    3|                456|
+-----+-------------------+



In [0]:
#h) Show the top 10 busiest airport. [Hint: Sum Total of Arrival and Departures from an Airport].
spark.sql("SELECT AIRPORT,SUM(TOTAL_FLIGHT_COUNT) AS TOTAL_FLIGHT_COUNT FROM( SELECT ORIGIN_AIRPORT as AIRPORT,count(*) AS TOTAL_FLIGHT_COUNT,'ORIGIN' from Airlines_Data group by ORIGIN_AIRPORT union all SELECT DESTINATION_AIRPORT,count(*),'DEST' from Airlines_Data group by DESTINATION_AIRPORT)A GROUP BY AIRPORT ORDER BY TOTAL_FLIGHT_COUNT DESC limit 10").show()

+-------+------------------+
|AIRPORT|TOTAL_FLIGHT_COUNT|
+-------+------------------+
|    ATL|              7220|
|    ORD|              5655|
|    DFW|              5553|
|    DEN|              4088|
|    LAX|              4028|
|    IAH|              3228|
|    PHX|              3166|
|    SFO|              3031|
|    LAS|              2790|
|    DTW|              2374|
+-------+------------------+



In [0]:
#Show the airlines that make the maximum number of cancellations.
spark.sql("SELECT AIRLINE,SUM(CANCELLED) as total_cancellations from Airlines_Data group by AIRLINE order by total_cancellations desc").show(1)

+-------+-------------------+
|AIRLINE|total_cancellations|
+-------+-------------------+
|     MQ|                414|
+-------+-------------------+
only showing top 1 row



In [0]:
#j) Find and order airlines in descending that make the most number of diversions.
spark.sql("SELECT AIRLINE,SUM(DIVERTED) as total_diversions from Airlines_Data GROUP BY AIRLINE order by total_diversions desc").show()

+-------+----------------+
|AIRLINE|total_diversions|
+-------+----------------+
|     WN|              35|
|     OO|              25|
|     EV|              22|
|     DL|              18|
|     B6|              16|
|     AA|              12|
|     US|               9|
|     UA|               8|
|     MQ|               5|
|     HA|               1|
|     NK|               0|
|     F9|               0|
|     AS|               0|
|     VX|               0|
+-------+----------------+



In [0]:
#k) Show days of month that seen the most number of diversion.
spark.sql("SELECT MONTH,SUM(DIVERTED) as total_diversions from Airlines_Data GROUP BY MONTH order by total_diversions desc").show(1)

+-----+----------------+
|MONTH|total_diversions|
+-----+----------------+
|    2|              60|
+-----+----------------+
only showing top 1 row



In [0]:
#l) Calculate mean and standard deviation of departure delay for all flights in minutes.
spark.sql("SELECT FLIGHT_NUMBER,avg(DEPARTURE_DELAY) as mean,STDDEV(DEPARTURE_DELAY) as standard_deviation FROM Airlines_Data GROUP BY FLIGHT_NUMBER").show()

+-------------+-------------------+------------------+
|FLIGHT_NUMBER|               mean|standard_deviation|
+-------------+-------------------+------------------+
|         1088| 18.473684210526315| 32.19276733997635|
|         1238|               0.55|10.148761915674868|
|         2866|            -3.5625| 3.285701345730213|
|         4935|              -11.0|              null|
|          148|  4.555555555555555|16.986105432905386|
|         4101|  7.277777777777778|19.167391290650087|
|         5803|               -4.0| 8.246211251235321|
|         6397|-1.3333333333333333| 7.094598884597588|
|          833| 13.303030303030303|  53.2455895171638|
|         2142|  8.909090909090908| 19.57270827174689|
|         4519| 18.166666666666668|45.657054949554805|
|         1591|  4.230769230769231| 9.976253857317436|
|         4900|             6.8125| 25.64428136381807|
|         2659|               -3.0|              null|
|         1342|               -6.0|3.5355339059327378|
|         

In [0]:
#m) Calculate mean and standard deviation of arrival delay for all flights in minutes.
spark.sql("SELECT FLIGHT_NUMBER,avg(ARRIVAL_DELAY) as mean,STDDEV(ARRIVAL_DELAY) as standard_deviation FROM Airlines_Data GROUP BY FLIGHT_NUMBER").show()

+-------------+-------------------+------------------+
|FLIGHT_NUMBER|               mean|standard_deviation|
+-------------+-------------------+------------------+
|         1088|  10.68421052631579|31.722849513978872|
|         1238|              -3.35|  14.3647045511596|
|         2866|           -16.4375| 8.671553109641511|
|         4935|                7.0|              null|
|          148| 0.4444444444444444|13.010679374182493|
|         4101|0.16666666666666666| 25.30810146968753|
|         5803|               -3.8| 11.03177229641729|
|         6397|               -8.0| 13.74772708486752|
|          833| 10.212121212121213|50.842746272057816|
|         2142| 0.6363636363636364|19.288715495194214|
|         4519|               11.0| 53.19022466581619|
|         1591|               -1.0| 14.52583904633395|
|         4900|            -2.4375|25.468853527396952|
|         2659|               -2.0|              null|
|         1342|               -9.8|12.755391017134675|
|         

In [0]:
#n) Find all diverted Route from a source to destination Airport & which route is the most diverted.
spark.sql("SELECT ORIGIN_AIRPORT,DESTINATION_AIRPORT,count(*) as total_diversions from Airlines_Data where DIVERTED = 1 group by ORIGIN_AIRPORT,DESTINATION_AIRPORT order by total_diversions DESC" ).show()


+--------------+-------------------+----------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|total_diversions|
+--------------+-------------------+----------------+
|           IAH|                ASE|               2|
|           STT|                PHL|               2|
|           PHL|                SAN|               2|
|           HOU|                DAL|               2|
|           ORD|                ASE|               2|
|           JFK|                EGE|               2|
|           CLT|                IAH|               2|
|           JFK|                SEA|               2|
|           TPA|                LGA|               2|
|           ATL|                GSP|               1|
|           MCO|                PVD|               1|
|           FLL|                PVD|               1|
|           FLL|                BWI|               1|
|           COS|                ORD|               1|
|           SFO|                BOI|               1|
|           SBP|            

In [0]:
r'''#o) Finding AIRLINES with its total flight count, total number of flights arrival delayed by more
than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not
Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some
of Airlines &#39;AK&#39;, &#39;HI&#39;, &#39;PR&#39;, &#39;VI&#39; and arrange output in descending order by % of such count of
flights.'''
spark.sql("select AIRLINE,Total_FLIGHTS,arr_delay_30,sum(case when Total_FLIGHTS>=10 then arr_delay_30_percent else 0 end)/Total_FLIGHTS*100 as ARRIVAL_DELAY_PERCENTAGE from (SELECT AIRLINE,count(FLIGHT_NUMBER) as Total_FLIGHTS, SUM(case when ARRIVAL_DELAY > 30 then 1 else 0 end) as arr_delay_30,SUM(case when ARRIVAL_DELAY > 30 and DAY_OF_WEEK not in (6,7) then 1 else 0 end) as arr_delay_30_percent FROM Airlines_Data where AIRLINE not in ('AK','HI','PR','VI') GROUP BY AIRLINE)a group by AIRLINE,Total_FLIGHTS,arr_delay_30 order by ARRIVAL_DELAY_PERCENTAGE desc").show()

+-------+-------------+------------+------------------------+
|AIRLINE|Total_FLIGHTS|arr_delay_30|ARRIVAL_DELAY_PERCENTAGE|
+-------+-------------+------------+------------------------+
|     F9|          794|         198|      17.506297229219143|
|     MQ|         3502|         775|       17.16162193032553|
|     B6|         2548|         485|      14.128728414442701|
|     NK|         1048|         186|      13.263358778625955|
|     EV|         5916|         874|      11.240703177822853|
|     OO|         5708|         859|      11.089698668535389|
|     UA|         4701|         653|       10.57221867687726|
|     AA|         5250|         700|       9.219047619047618|
|     VX|          573|          67|       8.202443280977311|
|     US|         3925|         452|       7.898089171974522|
|     DL|         7989|         746|       7.410189009888597|
|     WN|        11738|        1235|        7.40330550349293|
|     AS|         1586|         100|        4.03530895334174|
|     HA